## Jason's to-do list:
  #### - April-June 2014, 2016, 2018
  #### - Histogram to understand the distribution of the data
  #### - Summary Table
  ##### - Ways to summarize the hourly wind data: 
  - Mean wind speed for a county in a given month  
  - Max wind speed for a county in a given month 
  - Number of times we had high wind speed (15+ m2/sec) 
  #### - ArcPro Map -- Spatial map of which places receive the most wind events and when?

### [Project Guidelines](https://docs.google.com/document/d/13_CxrlfxYHFuPd_BJ2xTrvWDS41ie3TAWVYSBK8_nhU/edit?pli=1)

### Spatial and temporal analysis of the wind speed and wind event data and its relationship with crop loss (COL data):
  #### - Since we know that crop loss is directly proportional to wind speed, we will try to explore the relationship here. 
  - What types of wind variables have high correlation with crop loss? 
  - Is it maximum wind speed, number of high wind speed events in a given month, or something else? 
  #### - By overlaying the wind speed over the crop cover data (raster data), we can see which crops these high wind events generally damage and when?
  #### - Since crop loss data is available at a monthly time-scale at county level, we need to aggregate the wind data at the county level. 
  #### - What kind of aggregation would you need to do to explore its relationship with crop loss? E.g. (just thoughts) the number of medium vs high wind events in a month for a particular county, the number of tornadoes in the county, etc. 


# Data Sources
[NOAA Local Climate Data (LCD)](https://www.ncei.noaa.gov/maps/lcd/)
[NOAA Storm Events (NCEI)](https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/)
[Copernicus Climate Data Store (CDS)](https://cds.climate.copernicus.eu/cdsapp#!/home)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import glob
import regex as re
import netCDF4 as nc
import rasterio
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf
import ipywidgets as widgets
import matplotlib.ticker as mticker

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE
from rasterio.transform import from_origin
from rasterstats import zonal_stats
from shapely.geometry import LineString
from matplotlib.path import Path
from matplotlib.colors import Normalize
from netCDF4 import Dataset
from IPython.display import display
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
# Set up Bokeh to display plots inline in the notebook.
output_notebook(INLINE)
hv.extension('bokeh')
%matplotlib inline

In [ ]:
# Path to the directory
directory = 'Data'

# Check if the directory exists
if os.path.isdir(directory):
    src_dir = directory
else:
    src_dir = None

print(src_dir)

In [ ]:
who

In [ ]:
# Project extents
extent_coords = {'min_lat': 36.998665, 'max_lat': 37.734463,
                 'min_lon': -95.964735, 'max_lon': -94.616789}

# Extent Filter Function that works with the variable 'extents_coords'
def filter_dataframe_on_extent(df, extent_coords, lat1, lon1, lat2, lon2):
    min_lat, max_lat = extent_coords['min_lat'], extent_coords['max_lat']
    min_lon, max_lon = extent_coords['min_lon'], extent_coords['max_lon']
    return df[
        ((df[lat1] >= min_lat) & (df[lat1] <= max_lat) &
         (df[lon1] >= min_lon) & (df[lon1] <= max_lon)) |
        ((df[lat2] >= min_lat) & (df[lat2] <= max_lat) &
         (df[lon2] >= min_lon) & (df[lon2] <= max_lon))
        ]

In [ ]:
# Load the county boundary shapefile
sixco_fn = os.path.join(src_dir, 'GIS_files/KS_six_co_bo.shp')
data = gpd.read_file(sixco_fn)
data.head()

In [ ]:
# Load the Storm event data


In [ ]:
# # import matplotlib.pyplot as plt
# # import cartopy.crs as ccrs
# # import cartopy.feature as cf
# # import numpy as np
# # from netCDF4 import Dataset
# 
# def ncplotter(mrc, file: str, dep_var: str, width=6, height=6,
#               ax_title="Colorbar", plt_title="Plot Name", temp=False, upper_bound=1000):
#     """
#     Function to plot a specified variable from a netCDF format file using matplotlib and cartopy libraries.
#     The plot's extent is defined by the global 'extent_coords' variable which is set near the beginnning for the main script, so that is can be called on by other functions in the project. 
#     # Project extents
#     extent_coords = {'min_lat': 36.998665, 'max_lat': 37.734463,
#                  'min_lon': -95.964735, 'max_lon': -94.616789} 
#     An optional temperature 
#     conversion can be applied if the 'temp' argument is set to True. The function filters out values 
#     above 'upper_bound' and finds the maximum of the remaining values for each (latitude, longitude) pair.
# 
#     Parameters:
#     ----------
#     mrc : cartopy.crs object
#         The Coordinate Reference System in which to plot the data.
#     file : str
#         Path to the netCDF format file.
#     dep_var : str
#         Name of the variable from the netCDF file to be plotted.
#     width : int, optional
#         Width of the plot in inches. Default is 6.
#     height : int, optional
#         Height of the plot in inches. Default is 6.
#     ax_title : str, optional
#         The title for the colorbar. Default is "Colorbar".
#     plt_title : str, optional
#         The title for the plot. Default is "Plot Name".
#     temp : bool, optional
#         If True, the function assumes the provided variable is temperature in Kelvin, 
#         and will convert the temperature to Fahrenheit before plotting. Default is False.
#     upper_bound : float, optional
#         The maximum allowed value for the 'dep_var' variable. All values above 'upper_bound' 
#         are filtered out before the maximum of the remaining values is found. Default is 1000.
# 
#     Returns:
#     -------
#     None: The function does not return any value, it displays the plot diagram.
# 
#     Author: Gloria Hope
#     Modified by: Jason Ehlenberger for extents functionality
#     """
# 
#  # Open NetCDF dataset
#     ds = Dataset(file)
# 
#     # Get variable data
#     dep = ds[dep_var][:]
#     longs = ds['lon'][:]
#     lats = ds['lat'][:]
# 
#     if temp:
#         dep = (dep - 273.15) * 9 / 5 + 32  # Convert Kelvin to Fahrenheit
# 
#     xs, ys, zs = [], [], []
# 
#     # filter data that have <=1000 values and get its maximum
#     for lat in range(len(lats)):
#         for lon in range(len(longs)):
#             total = dep[:, lat, lon]
#             # Apply upper_bound condition and remove NaNs
#             total = total[total <= upper_bound]
#             if len(total) > 0:
#                 maximum = max(total)
#                 xs.append(longs[lon])
#                 ys.append(lats[lat])
#                 zs.append(maximum)
# 
#     fig = plt.figure(figsize=(width, height), dpi=100, facecolor="none")
#     ax = fig.add_subplot(1, 1, 1, projection=mrc)
#     ax.stock_img()
#     ax.coastlines()
#     ax.add_feature(cf.STATES)
#     ax.set_extent([extent_coords['min_lon'], extent_coords['max_lon'],
#                extent_coords['min_lat'], extent_coords['max_lat']], crs=mrc)
# 
#     plt.scatter(xs, ys, c=zs, cmap='rainbow', marker='s')
#     cb = plt.colorbar()
#     cb.ax.set_title(ax_title)
#     plt.title(plt_title)
# 
#     gls = ax.gridlines(draw_labels=True, color='none')
#     gls.top_labels = False
#     gls.right_labels = False
# 
#     plt.show()


In [ ]:
# ncplotter(ccrs.Mercator(),
#           os.path.join(src_dir, 'Gridmet_WIND/vs_2018.nc'),
#           'wind_speed',
#           10,
#           6,
#           'Maximum Daily Wind Speed',
#           'Maximum Daily Wind Speed in 2018',
#           False,1000)

In [ ]:
# import netCDF4 as nc
# import numpy as np

# Open the netCDF file
file = os.path.join(src_dir, 'Gridmet_WIND/wind_v_u_igust_2014.nc')
ds = nc.Dataset(file)

# Print out all variable names in the netCDF file
print(ds.variables.keys())

In [ ]:
# # Let's say you're interested in the variable named 'variable_of_interest'
# # You can print out information about this variable like this:
# print(ds.variables['time'])

In [ ]:
# # You can even read the data of variable
# data = np.array(ds.variables['time'])

In [ ]:
# # Now inspect the data. This code prints out the shape, min and max values of the array.
# # Replace data with zs if you managed to load it
# print(data.shape, np.nanmin(data), np.nanmax(data))
# 
# # Close the Dataset
# ds.close()

In [ ]:
# # import netCDF4 as nc
# # import numpy as np
# # import matplotlib.pyplot as plt
# # import cartopy.crs as ccrs
# # import matplotlib.ticker as mticker
# # from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
# 
# 
# def ncplotws(filename):
#     # Open netCDF file
#     dataset = nc.Dataset(filename)
# 
#     # Read variables
#     lon = dataset.variables['longitude'][:]
#     lat = dataset.variables['latitude'][:]
#     time = dataset.variables['time'][:]
#     u10 = dataset.variables['u10'][:]  # 10m u-component of wind
#     v10 = dataset.variables['v10'][:]  # 10m v-component of wind
#     i10fg = dataset.variables['i10fg'][:]  # Instantaneous 10m wind gust
# 
#     # Close the dataset
#     dataset.close()
# 
#     # Compute wind speed
#     wind_speed = np.sqrt(u10 ** 2 + v10 ** 2)
# 
#     # Plot Wind Speed
#     plt.figure(figsize=(10, 6))
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     plt.contourf(lon, lat, wind_speed[0, :, :], 60, transform=ccrs.PlateCarree())
# 
#     # Add geographic grid
#     gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
#     gl.top_labels = False
#     gl.right_labels = False
#     gl.xformatter = LONGITUDE_FORMATTER
#     gl.yformatter = LATITUDE_FORMATTER
# 
#     ax.coastlines()
# 
#     # Add title
#     ax.set_title('Wind Speed')
# 
#     plt.colorbar(label='Wind Speed (m/s)')
#     plt.show()
# 
# # Usage:
# # plot_wind_speed_from_netcdf('wind_v_u_igust_2014.nc')

In [ ]:
# ncplotws(os.path.join(src_dir, 'Gridmet_WIND/wind_v_u_igust_2014.nc'))

In [ ]:
# # import geopandas as gpd
# 
# 
# def ncplotws2(filename, shapefile):
#     # Open netCDF file
#     dataset = nc.Dataset(filename)
# 
#     # Read variables
#     lon = dataset.variables['longitude'][:]
#     lat = dataset.variables['latitude'][:]
#     time = dataset.variables['time'][:]
#     u10 = dataset.variables['u10'][:]  # 10m u-component of wind
#     v10 = dataset.variables['v10'][:]  # 10m v-component of wind
#     i10fg = dataset.variables['i10fg'][:]  # Instantaneous 10m wind gust
# 
#     # Close the dataset
#     dataset.close()
# 
#     # Compute wind speed
#     wind_speed = np.sqrt(u10 ** 2 + v10 ** 2)
# 
#     # Plot Wind Speed
#     plt.figure(figsize=(10, 6))
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     plt.contourf(lon, lat, wind_speed[0, :, :], 60, transform=ccrs.PlateCarree())
# 
#     # Add geographic grid
#     gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, color='blue', linewidth=1)
#     gl.top_labels = False
#     gl.right_labels = False
#     gl.xformatter = LONGITUDE_FORMATTER
#     gl.yformatter = LATITUDE_FORMATTER
# 
#     ax.coastlines()
# 
#     # Add title
#     ax.set_title('Wind Speed')
# 
#     # Overlay the shapefile
#     gdf = gpd.read_file(shapefile)
#     gdf.plot(ax=ax, facecolor="none", edgecolor='white', linewidth=2)
# 
#     plt.colorbar(label='Wind Speed (m/s)')
#     plt.show()

In [ ]:
# ncplotws2(os.path.join(src_dir, 'Gridmet_WIND/wind_v_u_igust_2014.nc'),sixco_fn)

In [ ]:
# import netCDF4 as nc
# import numpy as np
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# import ipywidgets as widgets
# import geopandas as gpd
# import matplotlib.ticker as mticker
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
# from scipy.interpolate import griddata


def ncplotws3(filename, shapefile):
    dataset = nc.Dataset(filename)
    lon = dataset.variables['longitude'][:]
    lat = dataset.variables['latitude'][:]
    time_units = dataset.variables['time'].units
    time_cal = dataset.variables['time'].calendar
    time = nc.num2date(dataset.variables['time'][:], time_units, time_cal)
    u10 = dataset.variables['u10'][:]
    v10 = dataset.variables['v10'][:]
    i10fg = dataset.variables['i10fg'][:]
    dataset.close()

    def plot_netcdf_time(time_index):
        wind_speed = np.sqrt(u10[time_index, :, :] ** 2 + v10[time_index, :, :] ** 2)
        plt.figure(figsize=(15, 5))
        ax = plt.axes(projection=ccrs.PlateCarree())
        plt.contourf(lon, lat, wind_speed, 60, transform=ccrs.PlateCarree())
    
        # Due to the possible large size of the data, we take every nth data point to keep the plot from becoming overloaded
        n = 1
        ax.barbs(lon[::n], lat[::n], u10[time_index, ::n, ::n], v10[time_index, ::n, ::n], length=5, color='white')

        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, color='blue', linewidth=1)
        gl.top_labels = False
        gl.right_labels = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        ax.coastlines()
        ax.set_title('Wind Speed and direction ' + str(time[time_index]))
        gdf = gpd.read_file(shapefile)
        gdf.plot(ax=ax, facecolor="none", edgecolor='white', linewidth=2)

        plt.colorbar(label='Wind Speed (m/s)')
        plt.show()

    time_slider = widgets.IntSlider(min=0, max=len(time) - 1, step=1, value=0, description='Time Index:')
    play = widgets.Play(min=0, max=len(time) - 1, step=24, value=0, interval=1000)
    widgets.jslink((play, 'value'), (time_slider, 'value'))
    
    plot_func = widgets.interactive_output(plot_netcdf_time, {'time_index': time_slider})
    
    display(widgets.HBox([play, time_slider]))
    display(plot_func)



In [ ]:
ncplotws3(os.path.join(src_dir, 'Gridmet_WIND/wind_v_u_igust_2014.nc'),sixco_fn)

In [ ]:
def ncplotws4(filename, boundary_shapefile, storm_shapefile, events_csv):
    dataset = nc.Dataset(filename)
    lon = dataset.variables['longitude'][:]
    lat = dataset.variables['latitude'][:]
    time_units = dataset.variables['time'].units
    time_cal = dataset.variables['time'].calendar
    time = nc.num2date(dataset.variables['time'][:], time_units, time_cal)
    u10 = dataset.variables['u10'][:]
    v10 = dataset.variables['v10'][:]
    i10fg = dataset.variables['i10fg'][:]
    dataset.close()

    def plot_netcdf_time(time_index):
        wind_speed = np.sqrt(u10[time_index, :, :] ** 2 + v10[time_index, :, :] ** 2)
        plt.figure(figsize=(15, 5))
        ax = plt.axes(projection=ccrs.PlateCarree())
        plt.contourf(lon, lat, wind_speed, 60, transform=ccrs.PlateCarree())

        n = 1
        ax.barbs(lon[::n], lat[::n], u10[time_index, ::n, ::n], v10[time_index, ::n, ::n], length=5, color='white')

        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, color='blue', linewidth=1)
        gl.top_labels = False
        gl.right_labels = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        ax.coastlines()

        ax.set_title('Wind Speed and Direction ' + str(time[time_index]))

        # Load and plot boundary shapefile
        boundary_gdf = gpd.read_file(boundary_shapefile)
        boundary_gdf.plot(ax=ax, facecolor="none", edgecolor='white', linewidth=2.0)

        # Load storm events shapefile and CSV, merge them (inner join) and plot
        storm_gdf = gpd.read_file(storm_shapefile)
        csv_df = pd.read_csv(events_csv, parse_dates=['BEGIN_DATE_TIME_det'])
        merged_gdf = storm_gdf.merge(csv_df, how='inner', left_on="EVENT_ID", right_on='EVENT_ID')

        for x in range(len(merged_gdf)):
            line = merged_gdf.geometry.iloc[x]
            ax.plot(line.x, line.y, color='green', linewidth=0.8)
            if x < 10:  # Adjust this threshold as needed
                ax.text(line.x[0], line.y[0], ' Event ' + str(merged_gdf.EVENT_ID.iloc[x]))

        plt.colorbar(label='Wind Speed (m/s)')
        plt.show()

    time_slider = widgets.IntSlider(min=0, max=len(time) - 1, step=1, value=0, description='Time Index:')
    play = widgets.Play(min=0, max=len(time) - 1, step=24, value=0, interval=1000)
    widgets.jslink((play, 'value'), (time_slider, 'value'))

    plot_func = widgets.interactive_output(plot_netcdf_time, {'time_index': time_slider})

    display(widgets.HBox([play, time_slider]))
    display(plot_func)


ncplotws4(os.path.join(src_dir, 'Gridmet_WIND/wind_v_u_igust_2014.nc'), 'sixco_fn', 'Data/Storm_event/StormEvents2014.shp',
          'Data/Storm_event/StormEvents_2014.csv')

In [ ]:
ncplotws4(os.path.join(src_dir, 'Gridmet_WIND/wind_v_u_igust_2014.nc'), 'sixco_fn', 'Data/Storm_event/StormEvents2014.shp',
          'Data/Storm_event/StormEvents_2014.csv')

In [42]:
# Read shapefile and relate table
shapefile = os.path.join(src_dir,'Storm_event/StormEvents2014.shp')
relate_table = os.path.join(src_dir, 'Storm_event/StormEvents_2014.csv')
gdf = gpd.read_file(shapefile)
relate_df = pd.read_csv(relate_table)
relate_df.head()

,EVENT_ID,EPISODE_ID,BEGIN_YEARMONTH_det,BEGIN_DAY_det,BEGIN_TIME_det,END_YEARMONTH_det,END_DAY_det,END_TIME_det,STATE_det,STATE_FIPS_det,...,DATA_SOURCE_det,YEARMONTH_loc,LOCATION_INDEX_loc,RANGE_loc,AZIMUTH_loc,LOCATION_loc,LATITUDE,LONGITUDE,LAT2,LON2
0,543659,90610,201410,9,2230,201410,10,1400,KANSAS,20,...,CSV,201410,1,1.41,WNW,MOUND VLY,37.2063,-95.4444,37.12378,-95.26664
1,543659,90610,201410,9,2230,201410,10,1400,KANSAS,20,...,CSV,201410,2,2.94,ENE,MOUND VLY,37.2102,-95.3682,37.12612,-95.22092
2,543659,90610,201410,9,2230,201410,10,1400,KANSAS,20,...,CSV,201410,3,3.27,SE,MOUND VLY,37.1699,-95.3740,37.10194,-95.22440
3,543659,90610,201410,9,2230,201410,10,1400,KANSAS,20,...,CSV,201410,4,2.90,SW,MOUND VLY,37.1699,-95.4568,37.10194,-95.27408
4,533649,88474,201409,5,1755,201409,5,1755,KANSAS,20,...,CSV,201409,1,3.83,SSW,KNIVETON,37.2700,-94.7000,37.16200,-94.42000


In [43]:
# get column names with their index
for i, col_name in enumerate(relate_df.columns):
    print(f"Index: {i}, Column Name: {col_name}")


Index: 0, Column Name: EVENT_ID
Index: 1, Column Name: EPISODE_ID
Index: 2, Column Name: BEGIN_YEARMONTH_det
Index: 3, Column Name: BEGIN_DAY_det
Index: 4, Column Name: BEGIN_TIME_det
Index: 5, Column Name: END_YEARMONTH_det
Index: 6, Column Name: END_DAY_det
Index: 7, Column Name: END_TIME_det
Index: 8, Column Name: STATE_det
Index: 9, Column Name: STATE_FIPS_det
Index: 10, Column Name: YEAR_det
Index: 11, Column Name: MONTH_NAME_det
Index: 12, Column Name: EVENT_TYPE_det
Index: 13, Column Name: CZ_TYPE_det
Index: 14, Column Name: CZ_FIPS_det
Index: 15, Column Name: CZ_NAME_det
Index: 16, Column Name: WFO_det
Index: 17, Column Name: BEGIN_DATE_TIME_det
Index: 18, Column Name: CZ_TIMEZONE_det
Index: 19, Column Name: END_DATE_TIME_det
Index: 20, Column Name: INJURIES_DIRECT_det
Index: 21, Column Name: INJURIES_INDIRECT_det
Index: 22, Column Name: DEATHS_DIRECT_det
Index: 23, Column Name: DEATHS_INDIRECT_det
Index: 24, Column Name: DAMAGE_PROPERTY_det
Index: 25, Column Name: DAMAGE_CROPS

In [46]:
# Drop the unwanted columns 
if len(relate_df.columns) > 25:
    rel_table = relate_df.drop(relate_df.columns[np.r_[2:8, 13:15, 16, 20:26, 27:29, 30:40, 41:43, 50:56]], axis=1)
else: rel_table = relate_df

# get column names with their index, again
for i, col_name in enumerate(rel_table.columns):
    print(f"Index: {i}, Column Name: {col_name}")

Index: 0, Column Name: EVENT_ID
Index: 1, Column Name: EPISODE_ID
Index: 2, Column Name: STATE_det
Index: 3, Column Name: STATE_FIPS_det
Index: 4, Column Name: YEAR_det
Index: 5, Column Name: MONTH_NAME_det
Index: 6, Column Name: EVENT_TYPE_det
Index: 7, Column Name: CZ_NAME_det
Index: 8, Column Name: BEGIN_DATE_TIME_det
Index: 9, Column Name: CZ_TIMEZONE_det
Index: 10, Column Name: END_DATE_TIME_det
Index: 11, Column Name: SOURCE_det
Index: 12, Column Name: FLOOD_CAUSE_det
Index: 13, Column Name: BEGIN_LOCATION_det
Index: 14, Column Name: END_LOCATION_det
Index: 15, Column Name: BEGIN_LAT_det
Index: 16, Column Name: BEGIN_LON_det
Index: 17, Column Name: END_LAT_det
Index: 18, Column Name: END_LON_det
Index: 19, Column Name: EPISODE_NARRATIVE_det
Index: 20, Column Name: EVENT_NARRATIVE_det
Index: 21, Column Name: LATITUDE
Index: 22, Column Name: LONGITUDE
Index: 23, Column Name: LAT2
Index: 24, Column Name: LON2


In [0]:
rel_table["BEGIN_DATE_TIME_det"] = pd.to_datetime(rel_table["BEGIN_DATE_TIME_det"], format="%d-%b-%y %H:%M:%S")

df = pd.merge(gdf, rel_table, on='EVENT_ID')
df.head(1)

In [ ]:
# import pandas as pd
# import geopandas as gpd
# from datetime import datetime
# 
# # Read shapefile and relate table
# shapefile = 'Output/StormEvents2014.shp'
# relate_table = os.path.join(src_dir, 'Storm_event/StormEvents_2014.csv')
# gdf = gpd.read_file(shapefile)
# relate_df = pd.read_csv(relate_table)
# relate_df["BEGIN_DATE_TIME_det"] = pd.to_datetime(relate_df["BEGIN_DATE_TIME_det"], format="%d-%b-%y %H:%M:%S")
# 
# df = pd.merge(gdf, relate_df, on='EVENT_ID')
# 
# # Checking if the merge is successful
# # print(df.shape)
# # print(df.head())
# 
# # Checking if there are any records after the filtering
# random_date = datetime.strptime("2014-01-01", "%Y-%m-%d").date()  # Example date
# df_time_filtered = df[df['BEGIN_DATE_TIME_det'].dt.date == random_date]
# # print(df_time_filtered.shape)
# # print(df_time_filtered.head())
# print(df['BEGIN_DATE_TIME_det'].dt.date.unique())


In [ ]:
# Call the function with your netCDF file and your shapefile
# widgets.interact(plot_slider_ws(os.path.join(src_dir, 'Gridmet_WIND/wind_v_u_igust_2014.nc'),'Output/StormEvents2014.shp')) #, os.path.join(src_dir, 'Storm_event/StormEvents_2014.csv')

In [ ]:
whos

Data munging steps in excel if the next function doesn't work.
For storm event csv add to column L this '=(LEFT(J1, 2) & "." & MID(J1, 3, LEN(J1)-2))*1', and in column M add '=(-LEFT(K1, 2) & "." & MID(K1, 3, LEN(K1)-2))*1'
Then copy down
Copy columns L and M and paste special 'values only' in L and M
delete comlumns J and K
change column headers to 'LAT2' and 'LON2'

In [ ]:
## Data munging function customized for Storm Event csv's from [NOAA Storm Events (NCEI)](https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/)

# def modify_csv(filename):
#     # Load the CSV data into a pandas DataFrame
#     df = pd.read_csv(filename)
# 
#     # Convert 'LAT2' and 'LON2' to string, insert decimal point after second digit, and then to floats:
#     if 'LAT2' in df.columns:
#         df['LAT2'] = (df['LAT2'].astype(str).str.lstrip('-').apply(lambda x: x[:2] + '.' + x[2:])).astype(float)
#     if 'LON2' in df.columns:
#         df['LON2'] = (df['LON2'].astype(str).str.lstrip('-').apply(lambda x: '-' + x[:2] + '.' + x[2:])).astype(float)
# 
#     
#     # Extract year from filename
#     match = re.search(r'_d(\d{4})', filename)
#     if match:
#         year = match.group(1)
#     else:
#         year = 'Unknown'
# 
#     # Construct new filename
#     old_filename = os.path.basename(filename)
#     
#     # Change 'd2014_c20231116' format to '2014'
#     year = re.search('d(\d{4})', old_filename).group(1)
#     
#     # Change 'StormEvents_locations-ftp_v1.0_d2014_c20231116.csv' format to 'StormEvents_locations'
#     new_part = old_filename.split('-')[0]
#     
#     # Construct new filename, change 'StormEvents_locations' format to 'StormEvents_locations_2014.csv'
#     new_filename = os.path.join(os.path.dirname(filename), f'{new_part}_{year}.csv')
# 
#     # Save modified DataFrame back to CSV
#     df.to_csv(new_filename, index=False)
#
#
# # Use glob.glob() to get all the files that start with 'Storm_event/StormEvents'
# files = glob.glob(os.path.join(src_dir, 'Storm_event/StormEvents_*'))
# 
# # Run the modify_csv function for each file using list comprehension
# for f in files:
#     modify_csv(f)

    event_ids_to_add = [498864, 508406, 508407, 508408, 508409, 508413, 508414, 508440, 508468, 508476, 508480, 508481, 508482, 516243, 516244, 516245, 516246, 526836, 526837, 533868, 542919, 543368, 543376, 543645, 543646, 543667, 627803, 627804, 627806, 630383, 630387, 632580, 633147, 636638, 655474, 659269, 659272, 659273, 659274, 659292, 660117, 660118, 661872, 661873, 662309, 662310, 663504, 663539, 663987, 663989, 663991, 754111, 755412, 756557, 756559, 756560, 756562, 756563, 756564, 756569, 756576, 756578, 756608, 756609, 756611, 756861, 765900, 765905, 772228, 772229, 774573, 774574, 779898, 779899, 780508, 780781, 787624, 787627, 787630, 792763]

In [ ]:
def join_and_save(src_dir, year, extent_coords=None):
    details = pd.read_csv(os.path.join(src_dir, f'Storm_event\StormEvents_details_{year}.csv'))
    locations = pd.read_csv(os.path.join(src_dir, f'Storm_event\StormEvents_locations_{year}.csv')).drop(
        columns='EPISODE_ID')

    # List of Event IDs to be kept even after extent filtering
    event_ids_to_add = [498864, 508406, 508407, 508408, 508409, 508413, 508414, 508440, 508468, 508476, 508480, 508481, 508482, 516243, 516244, 516245, 516246, 526836, 526837, 533868, 542919, 543368, 543376, 543645, 543646, 543667, 627803, 627804, 627806, 630383, 630387, 632580, 633147, 636638, 655474, 659269, 659272, 659273, 659274, 659292, 660117, 660118, 661872, 661873, 662309, 662310, 663504, 663539, 663987, 663989, 663991, 754111, 755412, 756557, 756559, 756560, 756562, 756563, 756564, 756569, 756576, 756578, 756608, 756609, 756611, 756861, 765900, 765905, 772228, 772229, 774573, 774574, 779898, 779899, 780508, 780781, 787624, 787627, 787630, 792763]

    # Separate out specific records
    details_to_add = details[details['EVENT_ID'].isin(event_ids_to_add)]
    locations_to_add = locations[locations['EVENT_ID'].isin(event_ids_to_add)]

    # Remove these specific records from details and locations before filtering
    details = details[~details['EVENT_ID'].isin(event_ids_to_add)]
    locations = locations[~locations['EVENT_ID'].isin(event_ids_to_add)]

    if extent_coords:
        details = filter_dataframe_on_extent(details, extent_coords, 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON')
        locations = filter_dataframe_on_extent(locations, extent_coords, 'LATITUDE', 'LONGITUDE', 'LAT2', 'LON2')

        # Append the separated records back to details and locations after filtering
        details = pd.concat([details, details_to_add])
        locations = pd.concat([locations, locations_to_add])

    details.columns = [col + '_det' if col not in ['EVENT_ID', 'EPISODE_ID'] else col for col in details.columns]
    locations.columns = [
        col + '_loc' if col not in ['EVENT_ID', 'EPISODE_ID', 'LATITUDE', 'LONGITUDE', 'LAT2', 'LON2'] else col for col
        in locations.columns]

    merged_df = pd.merge(details, locations, on='EVENT_ID', how='outer')

    column_order = ['EVENT_ID', 'EPISODE_ID'] + [col for col in merged_df.columns if
                                                 col not in ['EVENT_ID', 'EPISODE_ID']]
    merged_df = merged_df[column_order]

    merged_df.to_csv(os.path.join(src_dir, f'Storm_event\StormEvents_{year}.csv'), index=False)

for year in [2014, 2016, 2018]:
    join_and_save(src_dir, year, extent_coords = extent_coords)

def convert_dataframe_to_shapefile(source_file, output_file, extent_coords):
    """
    This function reads a CSV file and filters it based on latitude and longitude. 
    Then it drops NA rows from 'LATITUDE', 'LONGITUDE', 'LAT2', 'LON2' and converts the DataFrame into GeoDataFrame.
    Finally, the output is saved into shapefile format.
    
    Args:
    source_file: str: Path of the source CSV file.
    output_file: str: Path of the output shapefile.
    extent_coords: dict: 
        A dictionary containing the coordinates for area bounds to the filter. 
        Keys are 'min_lat', 'max_lat', 'min_lon', 'max_lon', belongs to either lon-lat pair. 

    Returns:
    None
    """

    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv(source_file, dtype={'AZIMUTH': str, 'LOCATION': str})

    # Pull out bounds for easier reference
    min_lat, max_lat = extent_coords['min_lat'], extent_coords['max_lat']
    min_lon, max_lon = extent_coords['min_lon'], extent_coords['max_lon']

    # Filter the records that either start or end within the given extents
    df_filtered = df[
        (
                (df['LATITUDE'] >= min_lat) &
                (df['LATITUDE'] <= max_lat) &
                (df['LONGITUDE'] >= min_lon) &
                (df['LONGITUDE'] <= max_lon)
        ) |
        (
                (df['LAT2'] >= min_lat) &
                (df['LAT2'] <= max_lat) &
                (df['LON2'] >= min_lon) &
                (df['LON2'] <= max_lon)
        )
        ]

    # Drop NA values from 'LATITUDE', 'LONGITUDE', 'LAT2', 'LON2'
    df_filtered = df_filtered.dropna(subset=['LATITUDE', 'LONGITUDE', 'LAT2', 'LON2'])

    # Create a new 'geometry' column in the DataFrame that contains LineString objects
    df_filtered['geometry'] = df_filtered.apply(lambda row: LineString(
        [(row['LONGITUDE'], row['LATITUDE']), (row['LON2'], row['LAT2'])]), axis=1)

    # Convert the DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(df_filtered, geometry='geometry')

    # Save the GeoDataFrame as a shapefile
    gdf.to_file(output_file)

source_file = os.path.join(src_dir, 'Storm_event/StormEvents2014.csv')
output_file = 'Output/StormEvents2014.shp'
extent_coords = {'min_lat': 36.998665, 'max_lat': 37.734463,
                 'min_lon': -95.964735, 'max_lon': -94.616789}

convert_dataframe_to_shapefile(source_file, output_file, extent_coords)

Exploratory Analysis
1) Describe the Data
2) Trends
3) Summary tables
4) drop unneeded columns
5) drop duplicates
6) drop outliers

